In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
# Load data
df = pd.read_csv('Ads_Clicks.csv')
df

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,0,0,1,0,0,0,0,1,0,0
9996,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0
9998,1,0,0,0,0,0,0,1,0,0


In [3]:
def mean(lst):
    return sum(lst)/len(lst)
def get_optimal(df,arms):
    max = sys.float_info.min
    optimal = 0
    count = 0
    for value in arms:
        if max < mean(df['Ad '+str(value)]):
            max = mean(df['Ad '+str(value)])
            optimal = count+1
        count += 1
    return optimal

In [4]:
def check(row,arm,reward):
    if(row[arm-1]==1):
        return reward
    return 0

In [5]:
def run_epsilon_greedy(df,time_steps,epsilon,reward=10):
    arms = np.arange(1,11)
    optimal = np.random.choice(arms)
    my_dict = {'Ad 1':[],'Ad 2':[],'Ad 3':[],'Ad 4':[],'Ad 5':[],'Ad 6':[],'Ad 7':[],'Ad 8':[],'Ad 9':[],'Ad 10':[]}
    total_reward = 0
    for i in range(time_steps):
        random_number = np.random.randint(0,100)
        row = df.iloc[i]
        select = optimal if random_number > 100*epsilon else np.random.choice(arms)
        total_reward += check(row,select,reward)
        for key in row.keys():
            my_dict[key].append(row[key])
        optimal = get_optimal(my_dict,arms)
    return total_reward

In [6]:
print("Epsilon -> 0.03, iterations -> 2000, Reward -> ",run_epsilon_greedy(df,2000,0.03))
print("Epsilon -> 0.3, iterations -> 2000, Reward -> ",run_epsilon_greedy(df,2000,0.3))

/tmp/ipykernel_50687/3798922512.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(row[arm-1]==1):


Epsilon -> 0.03, iterations -> 2000, Reward ->  5250
Epsilon -> 0.3, iterations -> 2000, Reward ->  4370


In [7]:
def update_mean(dict,means):
    for value in range(10):
        means[value] = (sum(dict['Ad '+str(value+1)])/len(dict['Ad '+str(value+1)]))
    return means

def update_std(dict,stds):
    for value in range(10):
        stds[value] = np.std(dict['Ad '+str(value+1)])
    return stds

def getMaxArm(stoch):
    max = sys.float_info.min
    count = 0
    index = 0
    for value in stoch:
        if value>max:
            max = value
            index = count
        count += 1
    return index

def run_upper_confidence_bound(df,time_steps,c,reward=10):
    arms = np.arange(1,11)
    means = np.full((10),0.0)
    stds = np.full((10),0.0)
    arms = np.arange(1,11)
    select = np.random.choice(arms)
    my_dict = {'Ad 1':[],'Ad 2':[],'Ad 3':[],'Ad 4':[],'Ad 5':[],'Ad 6':[],'Ad 7':[],'Ad 8':[],'Ad 9':[],'Ad 10':[]}
    total_reward = 0
    for i in range(time_steps):
        row = df.iloc[i]
        total_reward += check(row,select,reward)
        for key in row.keys():
            my_dict[key].append(row[key])
        means = update_mean(my_dict,means)
        stds = update_std(my_dict,stds)
        stochstic = []
        for i in range(10):
            stochstic.append(np.random.uniform(means[i]+c*stds[i],means[i]+c*stds[i]))
        select = getMaxArm(stochstic)+1
    return total_reward


In [9]:
print("Reward:")
run_upper_confidence_bound(df,2000,1.5)

Reward:


/tmp/ipykernel_50687/3798922512.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(row[arm-1]==1):


5360